In [ ]:
from selenium import webdriver
import pandas as pd
import csv
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup


In [ ]:
url = 'https://www.thegioididong.com/laptop-asus#c=44&m=122,128,119,120,118,203&o=17&pi=12' 
s = Service('E:\project\crawl-data\script\chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get(url)

In [ ]:
page_source = BeautifulSoup(driver.page_source)

In [ ]:
laptop_list = page_source.find_all('a',attrs={'class':'main-contain', 'data-box':'BoxCateFilter'})
link_list = []
for laptop in laptop_list:
    laptop_link = laptop.get('href')
    if laptop_link not in link_list:
        link_list.append('https://www.thegioididong.com' + laptop_link)
link_list
len(link_list)

In [64]:
names  = []
old_prices = []
present_prices = []
discounts = []
due_discounts = []
types = []   # Loại thiết bị
trademarks = [] # Thương hiệu
series = []    # Dòng sản phẩm
links = []

In [65]:
for link in link_list[:]:
    driver.get(link)
    sleep(10)
    page_source = BeautifulSoup(driver.page_source)
    try:
        #Lấy tên sản phẩm
        name = page_source.find('section', attrs={'data-cate-id':'44','class':'detail'}).find('h1').get_text().strip()
        
        #Lấy giá gốc của sản phẩm
        old_price = page_source.find('p',class_='box-price-old').get_text().strip() 
        
        #Lấy giá đã giảm của sản phẩm
        present_price = page_source.find('p',class_='box-price-present').get_text().strip() 
        
        #Lấy phần trăm discount của sản phẩm
        percent = page_source.find('p',class_='box-price-percent')
        discount = 'empty'
        if percent == None :
            discount = 'empty'
        else:
            discount = percent.get_text().strip()
            
        #Lấy hạn giảm giá
        due_discount = ' '.join(page_source.find('div', class_='pr-top').find('i',class_='pr-txt').get_text().strip().split(' ')[-3:])
        
        #Lấy loại thiết bị
        type = page_source.find('section', attrs={'data-cate-id':'44','class':'detail'}).find_all('li')[0].get_text().strip()
        
        #Lấy tên thương hiệu
        trademark = page_source.find('section', attrs={'data-cate-id':'44','class':'detail'}).find_all('li')[1].get_text().strip().split(' ')[1]
        
        #Lấy dòng sản phẩm
        temp = name.split(' ')
        serie = temp[temp.index(trademark) + 1]
        
        
        #Thêm các giá trị vào list tương ứng nếu không có lỗi
        names.append(name)
        old_prices.append(old_price)
        present_prices.append(present_price)
        discounts.append(discount)
        due_discounts.append(due_discount)
        types.append(type)
        trademarks.append(trademark)
        series.append(serie)
        links.append(link)
        
    except :
        pass

In [71]:
data = {'name' : names,\
        'old_price' : old_prices,\
        'present_price' : present_prices,\
        'discount' : discounts,\
        'due_discount' : due_discounts,\
        'type_of_device' : types,\
        'trademark' : trademarks,\
        'series' : series,\
        'link' : links
        }
df = pd.DataFrame(data)
df.head(87)


,name,old_price,present_price,discount,due_discount,type_of_device,trademark,series,link
0,Laptop HP Envy 16 h0205TX i9 12900H/32GB/512GB...,65.390.000₫,54.490.000₫ *,-16%,23:00 | 30/11,Laptop,HP,Envy,https://www.thegioididong.com/laptop/hp-envy-1...
1,Laptop Asus Gaming ROG Strix G16 G614JI i7 136...,60.990.000₫,53.990.000₫ *,-11%,23:00 | 30/11,Laptop,Asus,Gaming,https://www.thegioididong.com/laptop/asus-gami...
2,Laptop HP Omen 16 n0085AX R9 6900HX/32GB/1TB S...,62.890.000₫,52.190.000₫ *,-17%,23:00 | 30/11,Laptop,HP,Omen,https://www.thegioididong.com/laptop/hp-omen-1...
3,Laptop Dell XPS 13 Plus 9320 i5 1340P/16GB/512...,54.990.000₫,51.490.000₫,-6%,23:00 | 30/11,Laptop,Dell,XPS,https://www.thegioididong.com/laptop/dell-xps-...
4,Laptop Asus Gaming ROG Strix G16 G614JV i9 139...,52.990.000₫,47.990.000₫ *,-9%,23:00 | 30/11,Laptop,Asus,Gaming,https://www.thegioididong.com/laptop/asus-rog-...
...,...,...,...,...,...,...,...,...,...
82,Laptop HP Pavilion 15 eg2062TU i3 1215U/8GB/25...,14.990.000₫,12.190.000₫ *,-18%,23:00 | 30/11,Laptop,HP,Pavilion,https://www.thegioididong.com/laptop/hp-pavili...
83,Laptop Asus Vivobook 14 A1405VA i3 1315U/8GB/5...,14.990.000₫,12.490.000₫ *,-16%,23:00 | 30/11,Laptop,Asus,Vivobook,https://www.thegioididong.com/laptop/asus-vivo...
84,Laptop Lenovo V14 G4 IRU i3 1315U/8GB/512GB/Wi...,13.690.000₫,11.990.000₫ *,-12%,23:00 | 30/11,Laptop,Lenovo,V14,https://www.thegioididong.com/laptop/lenovo-v1...
85,Laptop Lenovo Ideapad 3 14ABA7 R5 5625U/8GB/25...,14.890.000₫,11.890.000₫ *,-20%,23:00 | 30/11,Laptop,Lenovo,Ideapad,https://www.thegioididong.com/laptop/lenovo-id...


In [72]:
df.to_csv('E:\project\crawl-data\outputed-csv-file\laptop_tgdd_50.csv',mode='a',index=False)